# Decision Trees
See also Random Forest.

DT training is a form of supervised learning.

A DT can be used for classification (categorical labels) or regession (numeric labels).
The DT can be trained on categorical or numerical features.
A DT can be binary, ternary, etc.  

Tree building is a form of inductive reasoning: 
it infers general rules from specific data.  
In fact, the general name for all DT algorithms is  
Top-Down Induction of Decision Trees (TDIDT).

A DT is a predictive model.
Prediction with a tree is a form of deductive reasoning:
it predicts a specific fact based on its general rules.

Each tree node could use any feature,
and for that feature, any threshold.
Thus, there are too many possible trees to examine them all.
Practical algorithms are greedy: 
they iteratively add the next best splitting rule.

DT decision boundaries are rectilinear, not curved as with most non-linear classifiers.

DT is less effective if there are feature interaction effects.
(Could we use tree nodes that test for combinations of features?
Probably, but the combinatorial explostion makes it impractical.)

High dimensionality slows training, especially if many features are irrelevant.  

A DT may be unusable if unseen data presents missing values
for features that were selected during training. A RF can overcome this.

Rules-based learning is a related concept. 
Rules-based learners are less strict about the order each rule is applied.
In contrast, a DT learns a rigid set of rules and an order for applying them.

## Hunt's algorithm
This was the first. This is the splitting criteria used by all the other algorithms listed here, including Quinlan.

Recursively split the remaining data based on the most discriminative feature.

Hunt's algorithm is optimal 
if all feature combinations are present in the training data.
Of course, this is unlikely in high-dimensional data.

## Quinlan's algorithms: 
Quinlan invented a series of algorithms. Each is an improvement on the previous.

#### ID3
ID stands for Iterative Dichotomizer. 
At each node, review the so-far unused features. 
Choose feature with max information gain or min entropy.
Choose one or more binary rules per node.
Each node directs its input to one of two or more child nodes.
Stop making rules when a node is out of examples, out of features, or the node is pure.
In a pure node, all the training samples that reach it are from the same class.

This is a greedy, non-optimal, recursive algorithm.
It tends to overfit. 
It can be improved, with longer run time, by backtracking (how?).
ID3 is better suited for classification with categorical features,  
than for regression with continuous features (why?).

#### FOIL
FOIL stands for First Order Inductive Learner algorithm. 
This is a hill climbing algorithm.
It adds one rule at a time.
The strategy is separate-and-conquer (not divide-and-conquer).

#### C4.5
This became world's most popular machine learning tool in the 1980s.
It was implemented as Weka J48.

Improvments relative to ID3:
* C4 is better at selecting thresholds on continuous features.
* C4 gnores missing data during its entropy calculation.
* C4, accepts weighted features.
* C4 prunes the final tree to remove unhelpful branches.

#### See5
This was commercial software.
* C5 requires less CPU and less RAM. 
* C5 makes smaller trees.
* C5 uses boosting (training sequentially on the residual cases).
* C5 uses feature selection (winnowing).
* C5 weights the different classification-error classes.

## Brieman's algorithms
#### CART
CART = Classification And Regression Tree.
CART was the acronym used by Brieman, but now it is a generic term.

#### Boosted trees
CART uses multiple trees in series (boosting) for incremental refinement.
This was used by AdaBoost and 
implemented by [XGBoost](https://xgboost.readthedocs.io/en/stable/tutorials/model.html).
It is implemented by sklearn as GradientBoostingClassifier.

It is computationally preferable to train many small trees in series rather than one big tree. 
Each tree is trained to classifiy residuals that the previous tree misclassified.
We say tree_2 classifies the residuals of tree_1.

Unclear to me: how boosted trees are used for prediction,
when you don't know whether the first tree's prediction was wrong.
Documentation says we combine the scores generated by the various models,
possibly using a weighted average, and possibly using trained weights.

#### Random Forests
RF reduces variance and avoids overitting.
RF trains and predicts with multiple trees in parallel.
Each tree is built on a random sample of the instances,
on a random sample of the features, or both.
Thus, each tree is imperfect, and the collection avoids overfitting.

RF uses bagging = boosted aggregation.
Each tree is built by sampling WITH replacement, called bootstrap.
This ensures that each tree is independent of the others.

## Impurity metrics
There are several impurity metrics
used by algorithms above to decide whether to split a node.  

In any DT, each node divides its training samples among its child nodes.
If the parent assigns one child instances of one class only, that child is pure.

Here, let $P(x_i)$ = proportion of examples in this node belonging to one class,
but we intend to sum over all classes.  

#### Entropy
Entropy = $\sum_i [ - P(x_i) * lg ( P(x_i) ) ] $   
Using base 2 logs, lg(1/2) = -1.   
Entropy is 0+0=0.0 at best, and $-[\frac{1}{2}(-1)+\frac{1}{2}(-1)]=1.0$ at worst.

#### Gini
Gini = $1 - \sum_i [ P(x_i)^2 ] $   
Gini is 1-(1+0)=0.0 at best, and $(0.5)^2+(0.5)^2=0.5$ at worst.

#### Classification error
Simple concept: a leaf "predicts" the majority class, so the rest is error.
There are many ways to say this:
* Classification error of node = 1 - (portion in largest class)  
* Classification error of node = (portion outside largest class), max = 0.5 for binary  
* Classification error of node = (total predicted wrong) / (training instances)

#### Information gain
You can measure the information gain of each node in a tree.  
Information gain = reduction of uncertainty = Entropy before split - Entropy after split.

#### FOIL information gain
FOIL adds one rule at a time until the information gain is too small.  
Say rule 0 finds $p_0$ positive and $n_0$ negative samples and $t_0=p_0+n_0$.  
Say rule 1 finds $p_1$ positive and $n_1$ negative samples and $t_1=p_1+n_1$.  
Determine whether the extra rule is worth it by:     
Gain = $p_1 [ lg(\frac{p_1}{t_1}) - lg(\frac{p_0}{t_0}) ]$

#### Collective impurity
The collective impurity of a node is the weighted sum of its children,
where each child's impurity is weighted by the portion of training instances it takes.  

## Example

In [1]:
# This node contains 5 of class1, 1 of class0.
# This node will predict the majority class, class1.
from math import log2
class0=1
class1=5
total=class0+class1
prob0=class0/total
prob1=class1/total
entropy = - ( prob0*log2(prob0) + prob1*log2(prob1) )
gini = 1 - (prob0**2 + prob1**2)
error = 1 - max(prob0,prob1)
print ('entropy:',entropy,'\ngini:',gini,'\nerror:',error)

entropy: 0.6500224216483541 
gini: 0.2777777777777777 
error: 0.16666666666666663


## Stopping
Decision trees tend to overfit. Possible solutions include:

### Stopping  
During training, stop splitting nodes when they are pure enough.
* Split till impurity is below threshold.  
* Equivalently, stop when splitting doesn't reduce impurity by a threshold amount.  
* Equivalently, stop when no remaining feature introduces a statistical change in the class distribution, based on t-test or chi-square test.     

### Pruning    
After training, go back and remove some leaf nodes.

Say the node has a 20:10 mix of 2 classes, so classification error = 10/30.  
The node has 4 leaf nodes with combined error = 9/30.  
The node was split because the split reduced total error.
But was it worth it?

Compute pessimistic error by adding a penalty of e.g. 0.5 per node.   
Then parent error = 10.5/30 but leaf error = (9+2)/30 = 11/30.
Seen this way, the split added error. 
So, prune this split.  